# Агентный RAG с Autogen

В этом ноутбуке показана реализация генерации с дополнением извлечением (RAG) с использованием агентов Autogen, обладающих расширенными возможностями оценки.


Исправление версии SQLite  
Если вы столкнулись с ошибкой:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Раскомментируйте этот блок кода в начале вашего ноутбука:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## Создание клиента

Сначала мы инициализируем клиент Azure AI Chat Completion. Этот клиент будет использоваться для взаимодействия с сервисом Azure OpenAI с целью генерации ответов на запросы пользователей.


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## Инициализация базы данных Vector

Мы инициализируем ChromaDB с постоянным хранилищем и добавляем улучшенные образцы документов. ChromaDB будет использоваться для хранения и извлечения документов, которые обеспечивают контекст для генерации точных ответов.


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## Реализация провайдера контекста

Класс `ContextProvider` отвечает за получение и интеграцию контекста из нескольких источников:

1. **Извлечение из векторной базы данных**: Использует ChromaDB для выполнения семантического поиска по туристическим документам
2. **Информация о погоде**: Поддерживает симулированную базу данных погоды для крупных городов
3. **Единый контекст**: Объединяет данные из документов и информацию о погоде в единый комплексный контекст

Основные методы:
- `get_retrieval_context()`: Извлекает релевантные документы на основе запроса
- `get_weather_data()`: Предоставляет информацию о погоде для указанных мест
- `get_unified_context()`: Объединяет контекст из документов и данных о погоде для улучшенных ответов


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## Настройка агента

Мы настраиваем агентов поиска и помощника. Агент поиска специализируется на нахождении релевантной информации с использованием семантического поиска, а помощник генерирует подробные ответы на основе найденной информации.


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## Обработка запросов с помощью RAG

Мы определяем функцию `ask_rag`, чтобы отправить запрос ассистенту, обработать ответ и оценить его. Эта функция управляет взаимодействием с ассистентом и использует оценщик для измерения качества ответа.


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# Пример использования

Мы инициализируем оценщик и определяем запросы, которые хотим обработать и оценить.


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## Запуск скрипта

Мы проверяем, выполняется ли скрипт в интерактивной среде или как стандартный скрипт, и запускаем основную функцию соответствующим образом.


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
